In [5]:
import pandas as pd


# Load datasets for fight finish analysis
df_finish_freq = pd.read_excel("fight_finish_quant.xlsx")  # Finish method frequency data
df_fighter = pd.read_csv("ufc_fight_data.csv")  # Fighter-level data
df_fights = pd.read_csv("ufc_fight_stat_data.csv")  # Fight statistics dataset
df_clean_fighters = pd.read_csv("clean_df_fighters.csv")  # Cleaned fighter dataset

In [9]:
# Merge fighter data with fight statistics using 'fight_id'
merged_fight_id = pd.merge(df_fighter, df_fights, on="fight_id", how="left")

# Merge with cleaned fighter dataset on 'fighter_id'
merged_fight_id = pd.merge(merged_fight_id, df_clean_fighters, on="fighter_id", how="left")

# Filter for male fighters only
male_fighters = merged_fight_id[merged_fight_id["gender"] != "F"]

# Display available columns in the merged dataset
print(male_fighters.columns)

Index(['fight_id', 'event_id', 'referee', 'f_1', 'f_2', 'winner', 'num_rounds',
       'title_fight', 'weight_class_x', 'gender', 'result', 'result_details',
       'finish_round', 'finish_time', 'fight_url_x', 'fight_stat_id',
       'fighter_id', 'knockdowns', 'total_strikes_att', 'total_strikes_succ',
       'sig_strikes_att', 'sig_strikes_succ', 'takedown_att', 'takedown_succ',
       'submission_att', 'reversals', 'ctrl_time', 'fight_url_y', 'full_name',
       'fighter_nickname', 'fighter_height_cm', 'fighter_weight_lbs',
       'fighter_reach_cm', 'fighter_stance', 'fighter_dob', 'fighter_w',
       'fighter_l', 'fighter_d', 'fighter_nc_dq', 'fighter_url',
       'weight_class_y', 'total_fights', 'win_rate'],
      dtype='object')


In [10]:
# Merge with finish frequency dataset on 'result_details'
merged_with_quant = pd.merge(merged_fight_id, df_finish_freq, on="result_details", how="left")

# Display available columns in the merged dataset
print(merged_with_quant.columns)

# Group data by weight class and gender, counting occurrences
grouped_result_details_gender = merged_with_quant.groupby(["weight_class_x", "result_details", "gender"]).size().reset_index(name="count")

# Separate data for male and female fighters
male_results = grouped_result_details_gender[grouped_result_details_gender["gender"] == "M"]
female_results = grouped_result_details_gender[grouped_result_details_gender["gender"] == "F"]

Index(['fight_id', 'event_id', 'referee', 'f_1', 'f_2', 'winner', 'num_rounds',
       'title_fight', 'weight_class_x', 'gender', 'result', 'result_details',
       'finish_round', 'finish_time', 'fight_url_x', 'fight_stat_id',
       'fighter_id', 'knockdowns', 'total_strikes_att', 'total_strikes_succ',
       'sig_strikes_att', 'sig_strikes_succ', 'takedown_att', 'takedown_succ',
       'submission_att', 'reversals', 'ctrl_time', 'fight_url_y', 'full_name',
       'fighter_nickname', 'fighter_height_cm', 'fighter_weight_lbs',
       'fighter_reach_cm', 'fighter_stance', 'fighter_dob', 'fighter_w',
       'fighter_l', 'fighter_d', 'fighter_nc_dq', 'fighter_url',
       'weight_class_y', 'total_fights', 'win_rate', 'Unnamed: 0',
       'num_fights'],
      dtype='object')


In [11]:
# Exclude decision-based results (Unanimous, Split, Majority)
finishes_only = male_results[~male_results["result_details"].isin(["Unanimous", "Split", "Majority"])]

# Sort by weight class and count (descending within each weight class)
sorted_finishes = finishes_only.sort_values(by=["weight_class_x", "count"], ascending=[True, False])

# Get the most common finish per weight class
most_common_finishes = sorted_finishes.groupby("weight_class_x").head(1).reset_index(drop=True)

# Drop gender column since it's no longer needed
most_common_finishes = most_common_finishes.drop(columns=["gender"])

# Display results
print(most_common_finishes)

      weight_class_x             result_details  count
0       Bantamweight  Punch to Head At Distance    106
1       Catch Weight           Rear Naked Choke     12
2      Featherweight  Punch to Head At Distance    116
3          Flyweight           Rear Naked Choke     62
4        Heavyweight  Punch to Head At Distance    202
5  Light Heavyweight  Punch to Head At Distance    162
6        Lightweight           Rear Naked Choke    256
7       Middleweight  Punch to Head At Distance    150
8        Open Weight           Rear Naked Choke     12
9       Welterweight  Punch to Head At Distance    208
